In [1]:
import pandas as pd
import numpy as np

def calculate_transitions(df_from, df_to):
    """
    Calculate transitions between two timepoints.
    Returns counts of users/non-users who continued or changed their usage.
    """
    # Get participants who completed both surveys
    common_participants = set(df_from['participantId']).intersection(set(df_to['participantId']))
    
    # Create a dictionary to store transitions
    transitions = {
        'users_continued': 0,
        'users_stopped': 0,
        'nonusers_started': 0,
        'nonusers_continued': 0
    }
    
    # For each common participant, check their usage patterns
    for pid in common_participants:
        usage_from = df_from[df_from['participantId'] == pid]['Usage'].iloc[0]
        usage_to = df_to[df_to['participantId'] == pid]['Usage'].iloc[0]
        
        if usage_from == 'Yes':
            if usage_to == 'Yes':
                transitions['users_continued'] += 1
            else:
                transitions['users_stopped'] += 1
        else:  # usage_from == 'No'
            if usage_to == 'Yes':
                transitions['nonusers_started'] += 1
            else:
                transitions['nonusers_continued'] += 1
    
    # Calculate total users and non-users in first timepoint for percentages
    total_users = len(df_from[df_from['participantId'].isin(common_participants) & 
                             (df_from['Usage'] == 'Yes')])
    total_nonusers = len(df_from[df_from['participantId'].isin(common_participants) & 
                                (df_from['Usage'] == 'No')])
    
    # Add percentages
    if total_users > 0:
        transitions['users_continued_pct'] = f"{(transitions['users_continued'] / total_users * 100):.1f}%"
        transitions['users_stopped_pct'] = f"{(transitions['users_stopped'] / total_users * 100):.1f}%"
    else:
        transitions['users_continued_pct'] = "0.0%"
        transitions['users_stopped_pct'] = "0.0%"
        
    if total_nonusers > 0:
        transitions['nonusers_started_pct'] = f"{(transitions['nonusers_started'] / total_nonusers * 100):.1f}%"
        transitions['nonusers_continued_pct'] = f"{(transitions['nonusers_continued'] / total_nonusers * 100):.1f}%"
    else:
        transitions['nonusers_started_pct'] = "0.0%"
        transitions['nonusers_continued_pct'] = "0.0%"
    
    return transitions, len(common_participants)

def analyze_usage_patterns(df1, df2, df3):
    """
    Analyze usage patterns across three survey timepoints.
    """
    results = {}
    
    # Initial group counts
    group_counts = df1['Group'].value_counts()
    total_initial = len(df1)
    results.update({
        'initial_new': group_counts.get('new', 0),
        'initial_new_pct': f"{(group_counts.get('new', 0) / total_initial * 100):.1f}%",
        'initial_non': group_counts.get('non', 0),
        'initial_non_pct': f"{(group_counts.get('non', 0) / total_initial * 100):.1f}%",
        'initial_current': group_counts.get('current', 0),
        'initial_current_pct': f"{(group_counts.get('current', 0) / total_initial * 100):.1f}%"
    })
    
    # Group to Usage in df1
    for group in ['new', 'non', 'current']:
        group_data = df1[df1['Group'] == group]
        group_total = len(group_data)
        if group_total > 0:
            usage_counts = group_data['Usage'].value_counts()
            yes_count = usage_counts.get('Yes', 0)
            no_count = usage_counts.get('No', 0)
            results.update({
                f'{group}_to_yes': yes_count,
                f'{group}_to_yes_pct': f"{(yes_count / group_total * 100):.1f}%",
                f'{group}_to_no': no_count,
                f'{group}_to_no_pct': f"{(no_count / group_total * 100):.1f}%"
            })
    
    # Overall usage counts for each timepoint
    for i, df in enumerate([df1, df2, df3], 1):
        total = len(df)
        usage_counts = df['Usage'].value_counts()
        yes_count = usage_counts.get('Yes', 0)
        no_count = usage_counts.get('No', 0)
        results.update({
            f'df{i}_users': yes_count,
            f'df{i}_users_pct': f"{(yes_count / total * 100):.1f}%",
            f'df{i}_nonusers': no_count,
            f'df{i}_nonusers_pct': f"{(no_count / total * 100):.1f}%"
        })
    
    # Calculate transitions between timepoints
    transitions_1_2, common_count_1_2 = calculate_transitions(df1, df2)
    transitions_2_3, common_count_2_3 = calculate_transitions(df2, df3)
    
    # Update results with transitions
    results.update({
        'users_continued_1_2': transitions_1_2['users_continued'],
        'users_continued_1_2_pct': transitions_1_2['users_continued_pct'],
        'users_stopped_1_2': transitions_1_2['users_stopped'],
        'users_stopped_1_2_pct': transitions_1_2['users_stopped_pct'],
        'nonusers_started_1_2': transitions_1_2['nonusers_started'],
        'nonusers_started_1_2_pct': transitions_1_2['nonusers_started_pct'],
        'nonusers_continued_1_2': transitions_1_2['nonusers_continued'],
        'nonusers_continued_1_2_pct': transitions_1_2['nonusers_continued_pct'],
        
        'users_continued_2_3': transitions_2_3['users_continued'],
        'users_continued_2_3_pct': transitions_2_3['users_continued_pct'],
        'users_stopped_2_3': transitions_2_3['users_stopped'],
        'users_stopped_2_3_pct': transitions_2_3['users_stopped_pct'],
        'nonusers_started_2_3': transitions_2_3['nonusers_started'],
        'nonusers_started_2_3_pct': transitions_2_3['nonusers_started_pct'],
        'nonusers_continued_2_3': transitions_2_3['nonusers_continued'],
        'nonusers_continued_2_3_pct': transitions_2_3['nonusers_continued_pct']
    })
    
    # Print diagnostics
    print(f"\nDiagnostic Information:")
    print(f"Total participants in each timepoint: df1={len(df1)}, df2={len(df2)}, df3={len(df3)}")
    print(f"Common participants between df1 and df2: {common_count_1_2}")
    print(f"Common participants between df2 and df3: {common_count_2_3}")
    
    print("\nTransitions from df1 to df2:")
    for key in ['users_continued', 'users_stopped', 'nonusers_started', 'nonusers_continued']:
        print(f"{key}: {transitions_1_2[key]} ({transitions_1_2[key + '_pct']})")
    
    print("\nTransitions from df2 to df3:")
    for key in ['users_continued', 'users_stopped', 'nonusers_started', 'nonusers_continued']:
        print(f"{key}: {transitions_2_3[key]} ({transitions_2_3[key + '_pct']})")
    
    # Save to CSV
    pd.DataFrame([results]).to_csv('usage_patterns.csv', index=False)
    
    return results

In [2]:
df = pd.read_csv("/work/Processed Data/CloudResearch-merged_24.11.11_processed.csv")
df

,Unnamed: 0.1,Unnamed: 0,index,StartDate,EndDate,Status_x,Progress,Duration (in seconds),Finished,RecordedDate,...,Use Interest_7_TEXT,Which Chatbots_S,Which Chatbots_S_3_TEXT,Chatbot Frequency_S,Chatbot Frequency_S_8_TEXT,Session Length_S,Usage Timeframe_S,Chatbot Device_S,Chatbot Device_S_4_TEXT,Interview_Interest
0,0,0,2,2024-06-10 16:25:22,2024-06-10 16:36:39,IP Address,100,677,True,2024-06-10 16:36:40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,3,2024-06-10 16:22:15,2024-06-10 16:37:20,IP Address,100,905,True,2024-06-10 16:37:21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,4,2024-06-10 16:26:47,2024-06-10 16:39:38,IP Address,100,770,True,2024-06-10 16:39:38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,5,2024-06-10 16:22:16,2024-06-10 16:39:57,IP Address,100,1060,True,2024-06-10 16:39:57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,6,2024-06-10 16:25:01,2024-06-10 16:40:11,IP Address,100,909,True,2024-06-10 16:40:11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,1460,352,373,2024-11-07 08:43:14,2024-11-07 08:57:44,IP Address,100,869,True,2024-11-07 08:57:45,...,NaN,Replika,NaN,Once a month or less,NaN,<15 min,a few weeks,"Computer,Smartphone",NaN,No
1461,1461,353,375,2024-11-07 14:39:52,2024-11-07 15:09:35,IP Address,100,1782,True,2024-11-07 15:09:36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
1462,1462,354,376,2024-11-08 07:46:39,2024-11-08 07:53:48,IP Address,100,428,True,2024-11-08 07:53:48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
1463,1463,355,377,2024-11-08 16:14:44,2024-11-08 16:27:05,IP Address,100,740,True,2024-11-08 16:27:06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes


In [3]:
df1 = df.loc[df['Survey_Number'] == 1]
df2 = df.loc[df['Survey_Number'] == 2]
df3 = df.loc[df['Survey_Number'] == 3]

In [4]:
results = analyze_usage_patterns(df1, df2, df3)
results


Diagnostic Information:
Total participants in each timepoint: df1=674, df2=434, df3=357
Common participants between df1 and df2: 433
Common participants between df2 and df3: 352

Transitions from df1 to df2:
users_continued: 186 (76.9%)
users_stopped: 56 (23.1%)
nonusers_started: 58 (30.4%)
nonusers_continued: 133 (69.6%)

Transitions from df2 to df3:
users_continued: 170 (85.0%)
users_stopped: 30 (15.0%)
nonusers_started: 33 (21.6%)
nonusers_continued: 119 (77.8%)


{'initial_new': 170,
 'initial_new_pct': '25.2%',
 'initial_non': 175,
 'initial_non_pct': '26.0%',
 'initial_current': 329,
 'initial_current_pct': '48.8%',
 'new_to_yes': 62,
 'new_to_yes_pct': '36.5%',
 'new_to_no': 108,
 'new_to_no_pct': '63.5%',
 'non_to_yes': 42,
 'non_to_yes_pct': '24.0%',
 'non_to_no': 133,
 'non_to_no_pct': '76.0%',
 'current_to_yes': 300,
 'current_to_yes_pct': '91.2%',
 'current_to_no': 29,
 'current_to_no_pct': '8.8%',
 'df1_users': 404,
 'df1_users_pct': '59.9%',
 'df1_nonusers': 270,
 'df1_nonusers_pct': '40.1%',
 'df2_users': 244,
 'df2_users_pct': '56.2%',
 'df2_nonusers': 190,
 'df2_nonusers_pct': '43.8%',
 'df3_users': 208,
 'df3_users_pct': '58.3%',
 'df3_nonusers': 149,
 'df3_nonusers_pct': '41.7%',
 'users_continued_1_2': 186,
 'users_continued_1_2_pct': '76.9%',
 'users_stopped_1_2': 56,
 'users_stopped_1_2_pct': '23.1%',
 'nonusers_started_1_2': 58,
 'nonusers_started_1_2_pct': '30.4%',
 'nonusers_continued_1_2': 133,
 'nonusers_continued_1_2_pct